In [12]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 

endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

endpoint_url = "https://query.wikidata.org/sparql"

query_img = """# Get images of artworks by Edvard Munch
SELECT ?item ?itemLabel ?image WHERE {
  # Find items that are artworks
  ?item wdt:P31 wd:Q41176.
  
  
  ?item wdt:P149 wd:Q994776.
  #?creator wdt:P136 wd:Q80113.

  #MINUS {?artwork wdt:P170 wd:Q41406.}
  # Retrieve the image property
  ?item wdt:P18 ?image.
  
  # Include labels for readability
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?itemLabel
LIMIT 50
"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

"""def get_text(textitem_id):
    q = ""
    if textitem_id:
        q = query_txt.replace("<placeholder>", "cps:"+textitem_id+" cpsp:P46 ?kuratorStatement.")
    else:
        q = query_txt.replace("<placeholder>","")
    results_txt = run_query(endpoint_url, q)
    for item in results_txt["results"]["bindings"]:
        print('Wikibase link: ' + '[' + item['textItem']['value'] + ']' + '(' + item['textItem']['value'] + ')' + '\n')
        print('Kurator: ' + item['kuratorLabel']['value'] + '\n')
        headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
        r = requests.get(item['textUrl']['value'], headers=headers, stream=True)
        text = str(r.text) #changed from r.content
        text = text.removeprefix("<!DOCTYPE html>") #changed from "b'<!DOCTYPE html>"
        print(markdownify(text)) #changed
"""
def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

def fetch_image_by_url(url, headers):
    r = requests.get(url, headers=headers, stream=True)
    if r.status_code == 200:
        im = Image.open(r.raw)
        return im
    if r.status_code == 500:
        return None
    if r.status_code == 403:
        return None
    if r.status_code == 429:
        timeout = get_delay(r.headers['retry-after'])
        print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
        time.sleep(timeout)
        fetch_image_by_url(url, headers)

def get_img(partOfItem_id):

    #print('<div class="img_grid" data-masonry="{ \"itemSelector\": \".img_small\" }"></div>')
    print('<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>')
    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    print('<div class="img_grid">')
    q = ""
    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)


    for item in results_img["results"]["bindings"]:    
      #print(item)
      #print('Wikibase link: ' + '[' + item['imgItem']['value'] + ']' + '(' + item['imgItem']['value'] + ')' + '\n')
      #print('Title: ' + item['itemLabel']['value'] + '\n')
      #print('Year: ' + item['publishDate']['value'] + '\n')
      #print('Description: ' + html.unescape(item['itemDescr']['value']) + '\n')

      # get image from image URL and display resized version


      image_url=item['image']['value']
      #headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
      #im = fetch_image_by_url(image_url, headers)
      #im.thumbnail((500, 500), Image.Resampling.LANCZOS)
      #display(im)
      onclickopen = f'openPopover("{image_url}")'
      onclickclose = f"closePopover('{image_url}')"
      print(f"<img class='img_small' src='{image_url}' popovertarget='{image_url}' onclick='{onclickopen}' >")
      #print(f'<input type="image" class="img_small" src="{image_url}" popovertarget="{image_url}" onclick="{onclickopen}" />')
      #print("<div class='fullimage'>")
      print(f'<img popover class="img_popover" src="{image_url}" id="{image_url}" onclick="{onclickclose}" style="display: none;">')    
      #print(f'<div class="close-btn" id="close_button_{image_url} onclick="{onclickclose}" style="display:none;"/></div>')
    print('<div class="grid-sizer"></div>')
    print('</div>')
    print("""
<script>
  var grid = document.querySelector('.img_grid');
  
var msnry = new Masonry( grid, {
  itemSelector: '.img_small',
  //columnWidth: '.grid-sizer',
  percentPosition: true
});

imagesLoaded( grid ).on( 'progress', function() {
  // layout Masonry after each image loads
  msnry.layout();
});

</script>

<script>
  function openPopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'block';
    // Prevent scrolling of the background
    //body.style.overflow = 'hidden';
    body.classList.add("stop-scrolling");
  }

  function closePopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
    body.classList.remove("stop-scrolling");
  }
          
</script>
          """)


def get_graph():
    import VizKG.visualize as vkg
    results_graph1 = run_query(endpoint_url, query_graph)
    #print(results_graph1)!
    #print('---')
    results_graph2 = run_query(endpoint_url, query_graph2)
    #print(results_graph2)

    chart = vkg(sparql_query=query_graph2, sparql_service_url=endpoint_url, chart='wordcloud')
    chart.plot()

In [13]:
#get_text("Q209")

In [14]:
get_img("")

<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>
<link rel='stylesheet' href='./masonrydesign.css'>
<div class="img_grid">
<img class='img_small' src='http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20the%20Czech%20and%20Slovak%20Republics%20in%20London%201.jpg' popovertarget='http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20the%20Czech%20and%20Slovak%20Republics%20in%20London%201.jpg' onclick='openPopover("http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20the%20Czech%20and%20Slovak%20Republics%20in%20London%201.jpg")' >
<img popover class="img_popover" src="http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20the%20Czech%20and%20Slovak%20Republics%20in%20London%201.jpg" id="http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20the%20Czech%20and%20Slovak%20Republics%20in%20London%201.jpg" onclick="closePopover('http://commons.wikimedia.org/wiki/Special:FilePath/Embassy%20of%20th

<hr>

In [15]:
#get_graph()